In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import cv2
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow import keras


tf.keras.backend.clear_session()  # For easy reset of notebook state.
print(cv2.__version__)
print(tf.__version__)
assert tf.executing_eagerly() == True

/home/charles/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/charles/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/charles/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/charles/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

4.1.2
1.14.0


In [2]:
IMG_WIDTH = 620
IMG_HEIGHT = 877
IMG_CHN = 3
NUM_F_POINTS = 5000
NUM_MATCHES = 500
BBOX_LENGTH = 21    

In [11]:
import numpy as np
import cv2

NUM_F_POINTS = 2000

# Batch processing of cv2 non-learning based method
def extract_features(ref_image, sns_image):
    # Create ORB detector with 5000 features. 

    ref_image = cv2.cvtColor(ref_image, cv2.COLOR_BGR2GRAY) 
    sns_image = cv2.cvtColor(sns_image, cv2.COLOR_BGR2GRAY) 
    orb_detector = cv2.ORB_create(5000) 
    kp1, d1 = orb_detector.detectAndCompute(ref_image, None) 
    kp2, d2 = orb_detector.detectAndCompute(sns_image, None) 
    return [kp1,kp2,d1,d2]

def extract_feature_batch(refs, sns):
    output = [[],[],[],[]]
    for i in range(refs.shape[0]):
        out = extract_features(refs[i], sns[i])
        for j in range(4):
            output[j].append(out[j])
    
    return output

def batch_match(d1s, d2s):
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
    matches = []

    for i in range(len(d1s)):
        matches.append(matcher.match(d1s[i], d2s[i]))
    return matches

def validate_match(matches):
    for i in range(len(matches)):
        matches[i].sort(key = lambda x: x.distance)
        matches[i] = matches[i][:int(len(matches)*60)]
    return matches

def calc_homographies(kp1s, kp2s, matches):
    # Define empty matrices of shape no_of_matches * 2. 
    homographies = []
    temp = matches
    matches = []

    for i in range(len(temp)):
        if temp[i] != []:
            matches.append(list(filter(None, temp[i])))
            
    matches = validate_match(matches)
    
    for i in range(len(matches)):
        p1 = np.zeros((len(matches[i]), 2)) 
        p2 = np.zeros((len(matches[i]), 2)) 
        if p1.shape[0] != 0:
            for j in range(len(matches[i])):
                p1[j, :] = kp1s[i][matches[i][j].queryIdx].pt 
                p2[j, :] = kp2s[i][matches[i][j].trainIdx].pt 
            homography, _ = cv2.findHomography(p1, p2, cv2.RANSAC) 
        else:
            homography = np.zeros([3,3])
        homographies.append(homography)
    return homographies

def register_images(sns_imgs, homographies, img_size = (IMG_WIDTH,IMG_HEIGHT), save = False):
    # Use this matrix to transform the 
    # colored image wrt the reference image. 
    transformed_imgs = []
    for i in range(sns_imgs.shape[0]):
        transformed_img = cv2.warpPerspective(sns_imgs[i], 
                            homographies[i], img_size) 
        if save: 
            cv2.imwrite('aligned_{}.jpg'.format(i), transformed_img) 
        
        transformed_imgs.append(transformed_img)
    return transformed_imgs

def visualize_matches(ref_imgs, sns_imgs, kp1s, kp2s, matches):
    for i in range(ref_imgs.shape[0]):
        imMatches = cv2.drawMatches(ref_imgs[i], kp1s[i], sns_imgs[i], kp2s[i], matches[i], None)
        cv2.imwrite("matches_{}.jpg".format(i), imMatches)
    
    
def get_alignment_matrix(kprs, kpss, drs, dss):
    '''
    Inputs
        kprs: F keypoints for each reference(query) image of shape N*F*3 with X,Y,Size
        kpss: F keypoints for each sensed(train) image of shape N*F*3 with X,Y,Size
        drs: F feature descriptors for each reference(query) image of shape N*F*32 
        dss: F feature descriptors for each sensed(train) image of shape N*F*32 
    Output
        aligned feature points and correlated distance of size N*f*7 X1,Y1,Size1, X2, Y2, Size2, Distance
    '''

    alignment_matrices = None
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
    for i in range(kprs.shape[0]):
        results = matcher.match(d1s[i], d2s[i])
        aligned = []
        for r in results:
            if len(aligned) >= NUM_MATCHES: break
            temp = np.concatenate((kprs[i][r.queryIdx], kpss[i][r.trainIdx]))
            aligned.append(np.concatenate((temp, [r.distance])))
            
        while len(aligned) < NUM_MATCHES:
            aligned.append([0,0,0,0,0,0,0])
            
        aligned = np.array([aligned])
        if alignment_matrices is None:
            alignment_matrices = aligned
        else:
            alignment_matrices = np.vstack((alignment_matrices, aligned))
            
    #sample alignment_matrix
    alignment_matrices = np.rint(alignment_matrices).astype(np.uint32)
    return alignment_matrices
    
    
def test_cv2_batch(dl):
    refs, sns = dl.load_image()
    kp1s, kp2s, d1s,d2s = extract_feature_batch(refs,sns)
    matches = batch_match(d1s, d2s)
    
    visualize_matches(refs, sns, kp1s, kp2s, matches)
    homos = calc_homographies(kp1s, kp2s, matches)
    imgs = register_images(sns, homos)
    for i in range(6):
            cv2.imwrite("registered_cv2{}.jpg".format(i), imgs[i])
        

    
def extract_feature_coordinates(ref_image, sns_image):
    # Create ORB detector with 5000 features. 

    ref_image = cv2.cvtColor(ref_image, cv2.COLOR_BGR2GRAY) 
    sns_image = cv2.cvtColor(sns_image, cv2.COLOR_BGR2GRAY) 
    orb_detector = cv2.ORB_create(NUM_F_POINTS) 
    kp1, d1 = orb_detector.detectAndCompute(ref_image, None) 
    kp2, d2 = orb_detector.detectAndCompute(sns_image, None) 
    kp1_np, kp2_np = [], []
    for i in range(NUM_F_POINTS):
        if i < len(kp1):
            kp1_np.append([kp1[i].pt[0],kp1[i].pt[1], kp1[i].size ] )
        else:
            kp1_np.append([0,0,0])
            d1 = np.vstack((d1, [np.zeros(32, dtype = np.uint8 )]))
            
        if i < len(kp2):
            kp2_np.append([kp2[i].pt[0],kp2[i].pt[1], kp2[i].size ] )
        else:
            kp2_np.append([0,0,0])
            d2 = np.vstack((d2, [np.zeros(32,dtype = np.uint8 )]))
        
    kp1_np, kp2_np = np.array(kp1_np) , np.array(kp2_np)

    return [kp1_np, kp2_np, d1, d2]

def extract_feature_coor_batch(refs, sns):
    output = []
    for i in range(refs.shape[0]):
        out = extract_feature_coordinates(refs[i], sns[i])
        for j in range(4):
            if len(output) < 4:
                output.append(np.expand_dims(out[j], axis=0))
            else: 
                output[j] = np.vstack( (output[j],np.expand_dims(out[j], axis=0)) )

    return output


def extract_match_patches(ref_imgs, sns_imgs, kprs, kpss, drs, dss):
    '''
    output: N * NUM_MATCHES * 2 * PATCH_H * PATCH_W * CHN Example:(6, 500, 2, 6, 6, 3)
    '''
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
    patches, matches = [], []
    for i in range(kprs.shape[0]):
        results = matcher.match(drs[i], dss[i])
        patch = []
        match = []
        for r in results:
            pair = []
            match_p = []
            if len(patch) >= NUM_MATCHES: break
            x, y, dis = kprs[i][r.queryIdx].astype(np.uint32)
            if  x-(BBOX_LENGTH-1) > 0 and y-(BBOX_LENGTH-1) > 0 \
                and x+(BBOX_LENGTH-1)/2 < IMG_WIDTH and y+(BBOX_LENGTH-1)/2 < IMG_HEIGHT:
                pair.append(ref_imgs[i][y-int((BBOX_LENGTH-1)/2):y+int((BBOX_LENGTH-1)/2)\
                                       ,x-int((BBOX_LENGTH-1)/2):x+int((BBOX_LENGTH-1)/2)])
                
            x, y, dis = kpss[i][r.trainIdx].astype(np.uint32)
            if  x-(BBOX_LENGTH-1) > 0 and y-(BBOX_LENGTH-1) > 0 \
                and x+(BBOX_LENGTH-1)/2 < IMG_WIDTH and y+(BBOX_LENGTH-1)/2 < IMG_HEIGHT:
                pair.append(sns_imgs[i][y-int((BBOX_LENGTH-1)/2):y+int((BBOX_LENGTH-1)/2)\
                                       ,x-int((BBOX_LENGTH-1)/2):x+int((BBOX_LENGTH-1)/2)])
                
            if len(pair) == 2:
                patch.append(np.array(pair))
                match.append(r)
                
        while len(patch) < NUM_MATCHES:
            patch.append(np.zeros((2,BBOX_LENGTH-1,BBOX_LENGTH-1, IMG_CHN)))
            match.append(None)
            
        patch = np.array(patch)
        patches.append(patch)
        match = np.array(match)
        matches.append(match)

    patches = np.array(patches)
    matches = np.array(matches)   

    return [patches[:,:,0,:,:,:], patches[:,:,1,:,:,:], matches]
    

def get_match_info(refs,sns):
    """
    returns in 255 scale
    """
    kprs,kpss, drs, dss = extract_feature_coor_batch(refs,sns)
    p_ref, p_sns, matches =  extract_match_patches(refs, sns, kprs, kpss, drs, dss)
    return [p_ref, p_sns, matches, kprs, kpss]
    
    
def get_model_inputs(refs,sns):
    p_ref, p_sns, matches, kprs, kpss = get_match_info(refs, sns)
    p_ref = (p_ref.astype(np.float32) / 255.0 ).reshape((p_ref.shape[0]*p_ref.shape[1],\
                                                        p_ref.shape[2], p_ref.shape[3], p_ref.shape[4]))
    p_sns = (p_sns.astype(np.float32) / 255.0).reshape((p_sns.shape[0]*p_sns.shape[1],\
                                                        p_sns.shape[2], p_sns.shape[3], p_sns.shape[4]))
    #matches = matches.reshape(matches.shape[0] * matches.shape[1])
    return [p_ref, p_sns, matches, kprs, kpss]
        
def visualize_corresponding_patches(p1, p2):
    for j in range(50):
        vis = (np.concatenate((p1[0][j], p2[0][j]), axis=1))
        cv2.imwrite("patch pair {}.jpg".format(j), vis)
        
def visualize_coords(img, c):
    for j in range(500):
        cv2.circle(img[0], (c[0][j][0], c[0][j][1]) , 1, (0, 0, 255), -1)
    cv2.imwrite("New feture img.jpg", img[0])


def patch_dist(p1,p2):

    return np.mean(((p1 - p2)**2)**0.5)

def get_central_coor(patch,img):
    
    W,H = patch.shape[0], patch.shape[1]
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            print("{} {}".format(i,j))
            if i+W < img.shape[0] and j+H < img.shape[1] and patch_dist(img[i:i+W, j:j+H, :],patch) < 1:
                return (i+W/2, j + H/2)
            
    print("patch does not exist in img.")
    return None


def patch_dist(p1,p2):

    return np.mean(((p1 - p2)**2)**0.5)

def get_central_coor(patch,img):
    
    W,H = patch.shape[0], patch.shape[1]
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            print("{} {}".format(i,j))
            if i+W < img.shape[0] and j+H < img.shape[1] and patch_dist(img[i:i+W, j:j+H, :],patch) < 1:
                return (i+W/2, j + H/2)
            
    print("patch does not exist in img.")
    return None


In [12]:
def get_model_inputs(refs,sns):
    p_ref, p_sns, matches, kprs, kpss = get_match_info(refs, sns)
    p_ref = (p_ref.astype(np.float32) / 255.0 ).reshape((p_ref.shape[0]*p_ref.shape[1],\
                                                        p_ref.shape[2], p_ref.shape[3], p_ref.shape[4]))
    p_sns = (p_sns.astype(np.float32) / 255.0).reshape((p_sns.shape[0]*p_sns.shape[1],\
                                                        p_sns.shape[2], p_sns.shape[3], p_sns.shape[4]))
    #matches = matches.reshape(matches.shape[0] * matches.shape[1])
    return [p_ref, p_sns, matches, kprs, kpss]


def generate_generator_multiple(generator, path, batch_size = 16, img_height = IMG_HEIGHT, img_width = IMG_WIDTH):

        gen_ref = generator.flow_from_directory(path,
                                              classes = ["ref"],
                                              target_size = (img_height,img_width),
                                              batch_size = batch_size,
                                              shuffle=False, 
                                              seed=7)

        gen_sns = generator.flow_from_directory(path,
                                              classes = ["sns"],
                                              target_size = (img_height,img_width),
                                              batch_size = batch_size,
                                              shuffle=False, 
                                              seed=7)
        while True:
                X1i = gen_ref.next()
                X2i = gen_sns.next()
                
                x,y,matches, _, _ = get_model_inputs(X1i[0].astype(np.uint8), X2i[0].astype(np.uint8))
                patch_input = np.concatenate((x,y), axis = 3)
                imgs = np.concatenate((X1i[0].astype(np.uint8), X2i[0].astype(np.uint8)), axis = 3)
                yield [patch_input,imgs], None #Yield both images and their mutual label

class Dataloader:
    def __init__(self, train_path, test_path, batch_size = 16):
        
        train_imgen = keras.preprocessing.image.ImageDataGenerator()
        test_imgen = keras.preprocessing.image.ImageDataGenerator()

        self.train_generator = generate_generator_multiple(generator=train_imgen,
                                               path = str(train_path),
                                               batch_size=batch_size)       

        self.test_generator = generate_generator_multiple(test_imgen,
                                              path = str(test_path),
                                              batch_size=batch_size)              

        
    def load_data(self, test = False):
        if test:
            return next(self.test_generator)
        else:
            return next(self.train_generator)
    
    def load_dl(self):
        return [self.train_generator, self.test_generator]
         

In [13]:
DIST_THRES = 100
RANDOM_PROB = 0.15
eps = 0.9
lr = 5e-4


def register_with_predicted(pred, matches, refs, snss):
    inds = np.argsort(pred, axis = 1)
    
    if np.random.uniform() < RANDOM_PROB * eps:#**(self.iteration / 100):
        inds = np.random.shuffle(inds)
    
    mask = np.where(inds < 50, np.ones_like(inds), np.zeros_like(inds)) # -1*NUM_MATCHES
    selected_matches = np.where( mask >= 0.5, matches, np.zeros_like(matches)  )
    good_matches = []
    
    for i in range(selected_matches.shape[0]):
        good_matches.append(selected_matches[i][selected_matches[i] != 0])

    kprs, kpss, _, _ =  extract_feature_batch(refs,snss)
    homo = calc_homographies(kprs, kpss, good_matches)
    imgs = np.array(register_images(snss, homo))
    
    return imgs, mask

def registration_loss(y_pred, imgs):

    
    ##############numpy operation###############
    refs, snss = imgs[:,:,:,:3], imgs[:,:,:,3:]
    _, _, matches, _, _ = get_model_inputs(refs, snss)
    

    pred = y_pred.numpy()
    imgs, mask = register_with_predicted(pred, matches, refs, snss)

    _, _, new_matches, kp1s, kp2s = get_match_info(refs, imgs)

    feature_diss,coor_diss, gt = [], [], []
    
    for i in range(kp1s.shape[0]):
        coor_dis, feature_dis, valid_count = 0, 0, 0
        for r in new_matches[i]:
            if r is None:
                continue
            valid_count+=1
            x1, y1, _ = kp1s[i][r.queryIdx]
            x2, y2, _ = kp1s[i][r.trainIdx]
            coor_dis += ((x1-x2)**2 + (y1-y2)**2)**0.5 #euc dist
            feature_dis += r.distance
            
        coor_diss.append(coor_dis/valid_count)
        feature_diss.append(feature_dis/valid_count)

    coor_diss = np.array(coor_diss)
    feature_diss = np.array(feature_diss)

    for c_d, f_d in zip(coor_diss, feature_diss):
        gt.append(np.ones_like(pred[0]) * 1/(c_d+f_d) )
    gt = np.array(gt)
    #################### END ####################
    #ground truth here: gt
    gt = tf.convert_to_tensor(gt)
    loss = tf.reduce_mean(((gt - y_pred)*mask)**2)
    
    return loss
    #def MSE(y_pred, y_true):
    #    loss = tf.reduce_mean(((gt - y_pred)*mask)**2)
    #    return loss
    
    #return MSE


In [14]:
class PatchRankingNet(tf.keras.Model):
    
    def __init__(self, dynamic=True):
        super(PatchRankingNet, self).__init__(dynamic=dynamic)
        self.conv1 = layers.Conv2D(16, 3 ,strides=(2, 2), name = "conv1",\
                                padding='valid', activation="relu", kernel_initializer='glorot_uniform')
        self.conv2 = layers.Conv2D(64, 3, strides=(1, 1), name = "conv2",\
                                padding='valid', activation="relu", kernel_initializer='glorot_uniform')
        self.conv3 = layers.Conv2D(128, 3, strides=(1, 1), name = "conv3",\
                                padding='valid', activation="relu", kernel_initializer='glorot_uniform')
        self.flat = layers.Flatten()
        self.fc1 = layers.Dense(32, activation= 'relu', name = "fc1", kernel_initializer = 'glorot_uniform')
        self.fc2 = layers.Dense(1, activation= 'sigmoid', name = "fc2", kernel_initializer = 'glorot_uniform')
        
    def call(self, inputs, training=True, mask=None):
        x, imgs = inputs
        imgs = imgs.numpy()
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flat(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = tf.reshape(x, (-1, NUM_MATCHES))
        self.add_loss(registration_loss(x,imgs))
        return x


#create DNN model


net = PatchRankingNet(dynamic = True)
net.compile(loss = None, optimizer =  keras.optimizers.Adam(learning_rate=lr), run_eagerly = True)

In [15]:
batch_size = 6
trainset_size = 6
testset_size = 6
epochs = 1000


dl = Dataloader(train_path = "./data/train", test_path= "./data/test", batch_size = batch_size)
train_generator, test_generator = dl.load_dl()
#net = patch_ranker()
net.fit_generator(train_generator,
                                steps_per_epoch=trainset_size/batch_size,
                                epochs = epochs,
                                validation_data = test_generator,
                                validation_steps = testset_size/batch_size,
                                use_multiprocessing = True,
                                shuffle = True)


Epoch 1/1000
Found 6 images belonging to 1 classes.
Found 6 images belonging to 1 classes.
Epoch 1/1000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Found 6 images belonging to 1 classes.
Found 6 images belonging to 1 classes.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
1/1 [==============================] - 14s 14s/step - loss: 0.0222 - val_loss: 0.0131
Epoch 2/1000
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)
(50, 2)


Exception in thread Thread-20:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/charles/.local/lib/python3.6/site-packages/tensorflow/python/keras/utils/data_utils.py", line 874, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "/home/charles/.local/lib/python3.6/site-packages/tensorflow/python/keras/utils/data_utils.py", line 866, in pool_fn
    initargs=(seqs, self.random_seed, get_worker_id_queue()))
  File "/usr/lib/python3.6/multiprocessing/context.py", line 119, in Pool
    context=self.get_context())
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 174, in __init__
    self._repopulate_pool()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 239, in _repopulate_pool
    w.start()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 1

KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/charles/.local/lib/python3.6/site-packages/tensorflow/python/keras/utils/data_utils.py", line 828, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "<ipython-input-12-c8b11e7ec3cc>", line 27, in generate_generator_multiple
    X1i = gen_ref.next()
  File "/home/charles/.local/lib/python3.6/site-packages/keras_preprocessing/image/iterator.py", line 116, in next
    return self._get_batches_of_transformed_samples(index_array)
  File "/home/charles/.local/lib/python3.6/site-packages/keras_preprocessing/image/iterator.py", line 230, in _get_batches_of_transformed_samples
    interpolation=self.interpolat

In [ ]:
patches, additionals = dl.load_data(test=True)
pred = net(patches)
refs,snss,matches = additionals
imgs = register_with_predicted(pred, matches, refs, snss)
for i in range(imgs.shape[0]):
    cv2.imwrite("registered_cv2{}.jpg".format(i), imgs[i])

In [15]:
        
def patch_ranker():
    
    patch_inputs = layers.Input(shape = (20, 20, 6))
    imgs_inputs = layers.Input(shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHN))

    x = layers.Conv2D(16, 3 ,strides=(2, 2), name = "conv1",\
                                    padding='valid', activation="relu", kernel_initializer='glorot_uniform')(patch_inputs)
    x = layers.Conv2D(64, 3, strides=(1, 1), name = "conv2",\
                                    padding='valid', activation="relu", kernel_initializer='glorot_uniform')(x)
    x = layers.Conv2D(128, 3, strides=(1, 1), name = "conv3",\
                                    padding='valid', activation="relu", kernel_initializer='glorot_uniform')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(32, activation= 'relu', name = "fc1", kernel_initializer = 'glorot_uniform')(x)
    out = layers.Dense(1, activation= 'sigmoid', name = "fc2", kernel_initializer = 'glorot_uniform')(x)

    net = keras.Model(inputs = [patch_inputs, imgs_inputs], outputs = out)

    net.compile(loss = registration_loss(out, imgs_inputs), optimizer = , run_eagerly = True)

    return net

SyntaxError: invalid syntax (<ipython-input-15-16547cebd750>, line 19)